# Session 6 - Classifying Cells (pt1)
*Goal: Learn how to make a Single Measurment Classifier and a ML object classifier to remove artifacts and find cells of interest*

## 1. Making Cell Measurements
We purposely turned off Make Meausurements in InstanSeg to save time by not calculating every possible metric. We're going to run some of the calculations now so that we can work with the cells

1. Select All Cells: `Objects > Select > Select detections > Select all detections`<br>
  <img src="Images/SelectAllDetections.PNG"><br>
2. Measure the average Hoechst intensity in every cell: `Analyze > Calculate Features > Add intensity features`. Then, use these settings:<br>
  <img src="Images/HoechstMeasurements.PNG"> <br>
3. Click <kbd>Run</kbd>. It will ask you if you want to "Process All Cells" (with other options in the dropdown). Click <kbd>OK</kbd>

## 2. Finding and removing anuclear cells - Single Measurement Classifier
InstanSeg bases the cell detection on the nuclear, cytoplasmic, and membrane signals simultaneously. Sometimes, it makes mistakes. In this sample, there are places where the autofluorescent fibers form a gap that looks almost like a cell membrane. But, there is no nucleus and we will not count it as a cell. 

  <img src="Images/AnuclearEx.PNG">
   
   We're going to get rid of these cells using the Hoechst measurement we just made. 
   
1. Create an **object classifier** to find cells lacks Hoechst. `Classify > Object classification > Create single measurement classifier`<br>
Make sure to **UNSELECT ALL CELLS** before checking the Live Preview. Selected objects always show up as a uniform color.
  1. On the Measurement dropdown, choose `ROI: 0.33 µm per pixel: Hoechst: Mean`
  2. On the Below threshold dropdown, choose "Other"
  3. Check Live Preview
  4. Adjust the threshold until you highlight *mostly* anuclear cells... and just a few "good" cells
  5. Name the classifier and save it
  
  <img src="Images/HoechstClassifier.gif">
 <br> 
 
2. Hit <kbd>Apply</kbd> and then close the classifier window

3. The default "Other" class color looks a lot like the yellow of selected cells. This can be confusing. To change the color of the Other cells, double click class name in the Annotation tab and pick a new color. 

  <img src="Images/ChangeClassColor.gif">


4. Save the file! Then, duplicate the project to preserve the raw data before you delete the artifacts, in case you change your mind. Go to your project folder, go one level up, and then copy-paste the entire project. Name the new copy "InstanSeg_All" or similar. 
  - At this point, you can also delete the version of LungImg2 with the watershed segmentation. In the Project tab, right click on 'LungImg2.ome.tif_Builtin cell detection' > `Remove Image`.  
 <img src="Images/RemoveImage.PNG">
 
5. Select all of the anuclear cells by going to the Annotations tab, right click on `Other`, then `Select Objects by Classification`
6. Delete the selected, "Other" cells. `Objects > Delete > Delete selected objects`


## 3. Finding Leukocytes - Training an ML Object Classifier
The Single Measurement Classifier works nicely for distinguishing cells with clear differences in 1 channel.  However, in many cases, it is more complicated to find the cells of interest. Machine Learning Classifiers to the rescue! The process of training an object classifier is similar to the process of training a [Pixel Classifier](./Session%203-%20Pixel%20Classifier.ipynb#2.-Train-a-Classifier)

1. Set up your [display settings](./Session%201-%20Visualization.ipynb#3.-Adjust-Visualization-Settings-) so that you can see: Hoechst, AF1, CD45, and S100a in different colors. Save these settings.
2. In the Annotations tab, in the class list, `right click > Add/Remove > Add class`

  <img src="Images/AddClass.PNG">
3. Create the 'CD45' class. Turn on Autoset.  
4. Use the brush tool to mark some CD45+ cells. The annotation must go over the center of the cell, but it does not need to completely cover the boundaries.  

  <img src="Images/BrushTraining.PNG">
  In this image, the cells marked in blue are included in the cyan CD45 training annotation, the cells marked in black are not. <br><br>
  
5. Also mark some CD45- cells with the Ignore* class. Make sure you include some Tumor cells and some CD45- stromal cells. In this example, we are going to focus on CD45 Bright cells. There are CD45 dim cells in the center of some tumor nodules, but we'll ignore them for now. 

6. So far, we only have Hoechst measurements on these cells. We need other measurements in other channels to identify CD45+ cells. 
  1. Select the cells ( `Objects > Select > Select detections > Select all detections`)
  2. Add intensity measurements (`Analyze > Calculate Features > Add intensity features`)
  3. Set the pixel size (0.325), check the AF1, CD45, and S100a channels, and check the Mean measurement
  4. Hit <kbd>Run</kbd>, then <kbd>OK</kbd>
7. Start training an Object Classifier: `Classify > Object classification > Train object classifier`
    <img src="Images/TrainObjectClassifier.PNG">
8. In the classifier training window, click <kbd>Live update</kbd>

    <img src="Images/LiveUpdate.PNG">
    
9. Just like in the pixel classifier workflow, iteratively improve the classifier by correcting mistakes.  When it's satisfactory, give the current version of the classifier a name and save it.  


## 4. Adding measurements to cells in multiple images
Just like with the pixel classifier, you cannot extrapolate an object classifier that was trained on 1 image to more samples. In order to train on the other samples in the project, **we must perform the same cell measurements on all of the images.** You can do this manually or through a script. Because there are only 3 images in this project, going one-by-one is reasonable. Or, learn to script it for the cases where you have hundreds!

#### Manual Method to repeat the measurements
  1. Open up one of the other two images you have not yet worked on.
  2. Select the cells and measure the mean intensities in the Hoechst, AF1, CD45, and S100a channels using 0.325 pixel size. (you can save time by doing this as a single step) as described in Step1. 
  3. Run the existing Anuclear classifier. 
    1. `Classify > Object classification > Load object classifier`. 
    2. Choose the Anuclear classifier and hit <kbd>Apply classifier</kbd>
  4. Select all "Other" cells (`right click > Select by classification`) and delete them (`Objects > Delete > Delete selected objects`)
  5. Repeat these steps on the 3rd image. 

#### Scripting to repeat the measurements
  1. Go to the Workflow tab and create a new Workflow
  2. Find the `Compute Intensity Features` where you measured Hoechst (Ch1). Right above it should be the line `Select objects by class`. Delete everything **above** this line. 
  
  <img src="Images/CleanUpMeasurementsp1.gif">
  
  3. Click <kbd>Create Script</kbd>
  
  4. Just like in [Session 5](./Session%205%20-%20Workflows%20to%20Scripts.ipynb#), we need to rearrange this draft script  to make it work. The order of the script
  should be:
    1. Select Detections
    2. Measure mean Hoescht intensity
    3. Run the "Anuclear" classifier
    4. Select "Other" cells
    5. Delete selected cells
    6. Select Detections
    7. Measure mean intensity in AF1, CD45, S100a
    
    Here is mine for reference:

In [ ]:
selectDetections();
runPlugin('qupath.lib.algorithms.IntensityFeaturesPlugin', '{"pixelSizeMicrons":0.325,"region":"ROI","tileSizeMicrons":25.0,"channel1":true,"channel2":false,"channel3":false,"channel4":false,"channel5":false,"channel6":false,"channel7":false,"channel8":falseerue,"channel9":false,"channel10":false,"channel11":false,"channel12":false,"channel13":false,"channel14":false,"channel15":false,"channel16":false,"channel17":false,"channel18":false,"channel19":false,"doMean":true,"doStdDev":false,"doMinMax":false,"doMedian":false,"doHaralick":false,"haralickMin":NaN,"haralickMax":NaN,"haralickDistance":1,"haralickBins":32}')
runObjectClassifier("Anuclear");
selectObjectsByClassification("Other");
clearSelectedObjects(true);
selectDetections();
runPlugin('qupath.lib.algorithms.IntensityFeaturesPlugin', '{"pixelSizeMicrons":0.325,"region":"ROI","tileSizeMicrons":25.0,"channel1":false,"channel2":true,"channel3":false,"channel4":false,"channel5":false,"channel6":false,"channel7":false,"channel8":false,"channel9":false,"channel10":false,"channel11":false,"channel12":false,"channel13":false,"channel14":false,"channel15":false,"channel16":true,"channel17":true,"channel18":false,"channel19":false,"doMean":true,"doStdDev":false,"doMinMax":false,"doMedian":false,"doHaralick":false,"haralickMin":NaN,"haralickMax":NaN,"haralickDistance":1,"haralickBins":32}')

  5. **Extra Credit assignment:** you can combine the two `runPlugin('qupath.lib.algorithms.IntensityFeaturesPlugin'` lines into a single command by finding all of the channels set to True the second time it is run, and changing them to True in the first version. Then, you'll measure all necessary channels at once and can delete the last 2 lines of the script entirely 
  
  6. Save the script as "CellMeasurements.groovy". Run the script for the two new images (`Run > Run for project`)

 ### 5. Combining training data

1. If you had closed the object classifier training window, open it again ( `Classify > Object classification > Train object classifier`)
2. Click <kbd>Load training</kbd><br>
<img src="Images/LoadObjClassTraining.PNG">
3. Press the double arrow <kbd>>></kbd> to include all project images in the training set. Then, click <kbd>Apply</kbd>
4. One at a time, add training annotations to each of the 3 images in the project. 
  - You want to mark at least a few CD45+ and CD45- cells in each image
  - Make sure the results are reasonably good in each image
5. Name the classifier "CD45" and <kbd>Save</kbd> it.
<br>
6. If you scripted the measurements, you can now add this line to the bottom of your script to apply the classifier: `runObjectClassifier("CD45")`

**You now have a script to find CD45+ leukocytes! **

### Immune infiltrate analysis
1. Calculate the frequency of leukocytes inside the tumor boundaries. Click on the Tumor annotation to read the area and the number of CD45 cells inside that region. 

<img src="Images/TumorImmuneInfiltrate.PNG">

  - The CD45 frequency is $NumCD45Cells / Area * 1000000 = cells / mm^2$
2. Perform this calculation for all 3 images. My results are:

<img src="Images/CD45permm.PNG">